## Universidad del Valle de Guatemala
## (CC3085) Inteligencia Artificial
## Laboratorio 6 - Árboles de Decisión

Miembros del equipo:
- Pedro Pablo Arriola Jiménez (20188)
- Oscar Fernando López Barrios (20679)
- Yong Bum Park (20117)
- Santiago Taracena Puga (20017)

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

### Task 1.1

In [3]:
data = pd.read_csv("./data/high_diamond_ranked_10min.csv")
# Ver las primeras filas del dataset
print(data.head())


       gameId  blueWins  blueWardsPlaced  blueWardsDestroyed  blueFirstBlood  \
0  4519157822         0               28                   2               1   
1  4523371949         0               12                   1               0   
2  4521474530         0               15                   0               0   
3  4524384067         0               43                   1               0   
4  4436033771         0               75                   4               0   

   blueKills  blueDeaths  blueAssists  blueEliteMonsters  blueDragons  ...  \
0          9           6           11                  0            0  ...   
1          5           5            5                  0            0  ...   
2          7          11            4                  1            1  ...   
3          4           5            5                  1            0  ...   
4          6           6            6                  0            0  ...   

   redTowersDestroyed  redTotalGold  redAvgLevel  

In [4]:
data.isna().sum()

gameId                          0
blueWins                        0
blueWardsPlaced                 0
blueWardsDestroyed              0
blueFirstBlood                  0
blueKills                       0
blueDeaths                      0
blueAssists                     0
blueEliteMonsters               0
blueDragons                     0
blueHeralds                     0
blueTowersDestroyed             0
blueTotalGold                   0
blueAvgLevel                    0
blueTotalExperience             0
blueTotalMinionsKilled          0
blueTotalJungleMinionsKilled    0
blueGoldDiff                    0
blueExperienceDiff              0
blueCSPerMin                    0
blueGoldPerMin                  0
redWardsPlaced                  0
redWardsDestroyed               0
redFirstBlood                   0
redKills                        0
redDeaths                       0
redAssists                      0
redEliteMonsters                0
redDragons                      0
redHeralds    

In [5]:
# División del dataset en entrenamiento, validación y prueba
X_trainval, X_test, y_trainval, y_test = train_test_split(data.drop(['blueWins'], axis=1), 
                                                          data['blueWins'], 
                                                          test_size=0.1, 
                                                          random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_trainval, 
                                                  y_trainval, 
                                                  test_size=0.1, 
                                                  random_state=42)



In [6]:
# Función para construir el árbol de decisión
class Node:
    def __init__(self, feature=None, threshold=None, value=None, left=None, right=None):
        self.feature = feature
        self.threshold = threshold
        self.value = value
        self.left = left
        self.right = right
        
def build_tree(X, y, max_depth=3, max_features=None, depth=0):
    # Si todos los elementos de y son iguales, devolvemos un nodo hoja con ese valor
    if len(y.unique()) == 1:
        return Node(value=y.iloc[0])
    
    # Si se alcanza la profundidad máxima, devolvemos un nodo hoja con la clase mayoritaria
    if depth == max_depth:  
        return Node(value=y.value_counts().idxmax())
    
    # Seleccionamos la mejor variable de separación y el umbral para dividir los datos
    best_feature, best_threshold = select_best_split(X, y, max_features)
    
    # Dividimos los datos en dos conjuntos, aquellos que cumplen la condición y aquellos que no
    left_idx = X[best_feature] < best_threshold
    right_idx = X[best_feature] >= best_threshold
    
    # Construimos los subárboles
    left = build_tree(X.loc[left_idx], y.loc[left_idx], max_depth,max_features, depth+1)
    right = build_tree(X.loc[right_idx], y.loc[right_idx], max_depth,max_features, depth+1)
    
    # Devolvemos el nodo raíz del subárbol
    return Node(feature=best_feature, threshold=best_threshold, left=left, right=right)

def select_best_split(X, y, max_features=None):
    n_features = X.shape[1]
    if max_features is not None and max_features < n_features:
        features = np.random.choice(X.columns, size=max_features, replace=False)
    else:
        features = X.columns
    
    best_feature, best_threshold, best_gini = None, None, 1.0
    
    for feature in features:
        for threshold in X[feature].unique():
            left_idx = X[feature] < threshold
            right_idx = X[feature] >= threshold
            left_gini = gini(y.loc[left_idx])
            right_gini = gini(y.loc[right_idx])
            gini_impurity = (left_gini * sum(left_idx) + right_gini * sum(right_idx)) / len(y)
            
            if gini_impurity < best_gini:
                best_feature = feature
                best_threshold = threshold
                best_gini = gini_impurity
                
    return best_feature, best_threshold


def gini(y):
    # Calculamos la proporción de cada clase en y
    p = y.value_counts(normalize=True)
    
    # Calculamos el índice de Gini
    gini = 1 - sum(p**2)
    
    return gini

def predict(x, tree):
    # Si llegamos a un nodo hoja, devolvemos la clase correspondiente
    if tree.value is not None:
        return tree.value
    
    # Si la instancia cumple la condición del nodo, continuamos por el subárbol izquierdo
    if x[tree.feature] < tree.threshold:
        return predict(x, tree.left)
    
    # Si no, continuamos por el subárbol derecho
    else:
        return predict(x, tree.right)


In [7]:
# Construir el árbol de decisión con el conjunto de entrenamiento
tree = build_tree(X_train, y_train, max_depth=4, max_features=5)
#en este

In [8]:
# Hacer predicciones con el conjunto de validación
y_pred = X_val.apply(lambda x: predict(x, tree), axis=1)

In [9]:
# Calcular la precisión de las predicciones
accuracy = accuracy_score(y_val, y_pred)
print(f'Precisión en el conjunto de validación: {accuracy:.3f}')

Precisión en el conjunto de validación: 0.730


In [10]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Cargar los datos
data = pd.read_csv("./data/high_diamond_ranked_10min.csv")

# División del dataset en entrenamiento, validación y prueba
X_trainval, X_test, y_trainval, y_test = train_test_split(data.drop(['blueWins'], axis=1), 
                                                          data['blueWins'], 
                                                          test_size=0.1, 
                                                          random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_trainval, 
                                                  y_trainval, 
                                                  test_size=0.1, 
                                                  random_state=42)

# Entrenar el modelo
clf = DecisionTreeClassifier(max_depth=3, max_features=5, random_state=42)
clf.fit(X_train, y_train)

# Hacer predicciones con el conjunto de validación
y_pred = clf.predict(X_val)

# Calcular la precisión de las predicciones
accuracy = accuracy_score(y_val, y_pred)
print(f'Precisión en el conjunto de validación: {accuracy:.3f}')


Precisión en el conjunto de validación: 0.734


0.709 segun el que hicimos
0.734 con el la libreria

### Task 1.2

In [14]:
data_fifa = pd.read_csv("./data/fifa_dataset.csv")
data_fifa

C:\Users\Fernando Lopez\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (23,35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,0,Cristiano Ronaldo,32,https://cdn.sofifa.org/48/18/players/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Real Madrid CF,https://cdn.sofifa.org/24/18/teams/243.png,...,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,1,L. Messi,30,https://cdn.sofifa.org/48/18/players/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,93,93,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,...,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,2,Neymar,25,https://cdn.sofifa.org/48/18/players/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,94,Paris Saint-Germain,https://cdn.sofifa.org/24/18/teams/73.png,...,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
3,3,L. Suárez,30,https://cdn.sofifa.org/48/18/players/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,92,92,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,...,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
4,4,M. Neuer,31,https://cdn.sofifa.org/48/18/players/167495.png,Germany,https://cdn.sofifa.org/flags/21.png,92,92,FC Bayern Munich,https://cdn.sofifa.org/24/18/teams/21.png,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17976,17976,A. Kelsey,17,https://cdn.sofifa.org/48/18/players/237463.png,England,https://cdn.sofifa.org/flags/14.png,46,63,Scunthorpe United,https://cdn.sofifa.org/24/18/teams/1949.png,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17977,17977,B. Richardson,47,https://cdn.sofifa.org/48/18/players/11728.png,England,https://cdn.sofifa.org/flags/14.png,46,46,Wycombe Wanderers,https://cdn.sofifa.org/24/18/teams/1933.png,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17978,17978,J. Young,17,https://cdn.sofifa.org/48/18/players/231381.png,Scotland,https://cdn.sofifa.org/flags/42.png,46,61,Swindon Town,https://cdn.sofifa.org/24/18/teams/1934.png,...,31.0,28.0,38.0,29.0,45.0,42.0,45.0,44.0,32.0,45.0
17979,17979,J. Lundstram,18,https://cdn.sofifa.org/48/18/players/238813.png,England,https://cdn.sofifa.org/flags/14.png,46,64,Crewe Alexandra,https://cdn.sofifa.org/24/18/teams/121.png,...,47.0,46.0,45.0,47.0,43.0,45.0,41.0,44.0,46.0,41.0


In [15]:
# División del dataset en entrenamiento, validación y prueba
X_train_validation, X_test, y_train_validation, y_test = train_test_split(data_fifa.drop(['Potential'], axis=1), data_fifa['Potential'], test_size=0.1, random_state=42)

X_train, X_validation, y_train, y_validation = train_test_split(X_train_validation, y_train_validation, test_size=0.1, random_state=42)

In [19]:
# Construir el árbol de decisión con el conjunto de entrenamiento
tree = build_tree(X_train, y_train, max_depth=1, max_features=1)